In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_curve, auc,classification_report

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from math import sqrt


plt.rcParams['font.sans-serif'] = ['simhei']
plt.rcParams['font.serif'] = ['simhei']
plt.rcParams['axes.unicode_minus']=False # 默认是使用Unicode负号，设置正常显示字符，如正常显示负号

import seaborn as sns
sns.set_style("darkgrid",{"font.sans-serif":['simhei','Droid Sans Fallback']})
import pandas as pd


In [3]:
data=pd.read_csv('A题-全部数据/附件2/M202.csv')
data.head()

日期  时间 生产线编号  物料推送气缸推送状态  物料推送气缸收回状态  物料推送数  物料待抓取数  放置容器数  容器上传检测数  填装检测数  \
0   1   0  M202           0           1      0       0      0        0      0   
1   1   1  M202           1           0      1       0      0        0      0   
2   1   2  M202           0           1      1       0      0        0      0   
3   1   3  M202           1           0      2       0      0        0      0   
4   1   4  M202           0           1      2       1      1        0      0   

   ...  不合格数  物料推送装置故障1001  物料检测装置故障2001  填装装置检测故障4001  填装装置定位故障4002  \
0  ...     0           NaN           NaN           NaN           NaN   
1  ...     0           NaN           NaN           NaN           NaN   
2  ...     0           NaN           NaN           NaN           NaN   
3  ...     0           NaN           NaN           NaN           NaN   
4  ...     0           NaN           NaN           NaN           NaN   

   填装装置填装故障4003  加盖装置定位故障5001  加盖装置加盖故障5002  拧盖装置定位故障6001  拧盖装置拧盖故障6002  
0           NaN           NaN           NaN           NaN           NaN  
1           NaN           NaN           NaN           NaN           NaN  
2           NaN           NaN           NaN           NaN           NaN  
3           NaN           NaN           NaN           NaN           NaN  
4           NaN           NaN           NaN           NaN           NaN  

[5 rows x 37 columns]

In [4]:
data.shape

(7528712, 37)

In [5]:
X =data[[ '时间', '物料推送气缸推送状态', '物料推送气缸收回状态', '物料推送数', '物料待抓取数',
       '放置容器数', '容器上传检测数', '填装检测数', '填装定位器固定状态', '填装定位器放开状态', '物料抓取数', '填装旋转数',
       '填装下降数', '填装数', '加盖检测数', '加盖定位数', '推盖数', '加盖下降数', '加盖数', '拧盖检测数',
       '拧盖定位数', '拧盖下降数', '拧盖旋转数', '拧盖数', '合格数', '不合格数']]

In [6]:
import joblib

# load the saved model
model = joblib.load('./modellist/Q1_物料推送装置故障1001.bin')

# predict using the loaded model
model.predict(X)

array([0, 0, 0, ..., 0, 0, 0])

In [7]:
X=np.array(X)

In [8]:
import pandas as pd
import joblib

# 假设 X 已经是一个合适的输入数据格式
# X = ...

# 加载模型
models = {
    "1001": joblib.load('./modellist/Q1_物料推送装置故障1001.bin'),
    "2001": joblib.load('./modellist/Q1_物料检测装置故障2001.bin'),
    "4003": joblib.load('./modellist/Q1_填装装置填装故障4003.bin'),
    "4001": joblib.load('./modellist/Q1_填装装置检测故障4001.bin'),
    "4002": joblib.load('./modellist/Q1_填装装置定位故障4002.bin'),
    "6002": joblib.load('./modellist/Q1_拧盖装置拧盖故障6002.bin'),
    "5002": joblib.load('./modellist/Q1_加盖装置加盖故障5002.bin'),
    "5001": joblib.load('./modellist/Q1_加盖装置定位故障5001.bin'),
    "6001": joblib.load('./modellist/Q1_拧盖装置定位故障6001.bin')
}

# 对新的输入数据 X 进行预测，并收集预测结果
predictions = {}
for code, model in models.items():
    predictions[code] = model.predict(X)




In [9]:
for i,j in predictions.items():
    data[i]=j
    print(i)
    print(data[i].value_counts())

1001
1001
0    7524463
1       4249
Name: count, dtype: int64
2001
2001
0    7523693
1       5019
Name: count, dtype: int64
4003
4003
0    7519463
1       9249
Name: count, dtype: int64
4001
4001
0    7526921
1       1791
Name: count, dtype: int64
4002
4002
0    7522633
1       6079
Name: count, dtype: int64
6002
6002
0    7518416
1      10296
Name: count, dtype: int64
5002
5002
0    7512445
1      16267
Name: count, dtype: int64
5001
5001
0    7517976
1      10736
Name: count, dtype: int64
6001
6001
0    7525242
1       3470
Name: count, dtype: int64


In [10]:
a=data[data['1001']==1]

In [11]:
# 根据用户的明确说明，连续的时间值表示持续的故障事件，
# 我们应该相应地将这些事件分组来计算持续时间。

# 提取唯一的日期，以便分别处理每天的数据
days = a['日期'].unique()

# 初始化一个空的DataFrame，用于存储处理后的结果
processed_data = pd.DataFrame(columns=['序号', '日期', '开始时间', '持续时长/秒'])

# 对每一天的数据进行处理，计算连续事件的持续时间
for day in days:
    # 筛选出当前天的数据，并按时间排序
    day_data = a[a['日期'] == day].sort_values(by='时间')
    
    # 初始化变量，用于跟踪连续事件
    start_time = None  # 开始时间
    last_time = None   # 上一个时间点
    duration = 0       # 持续时间
    
    # 用于存储当天事件的列表
    events = []
    
    for index, row in day_data.iterrows():
        if start_time is None:
            # 开始一个新的事件
            start_time = last_time = row['时间']
            duration = 1
        elif row['时间'] == last_time + 1:
            # 继续当前的事件
            last_time = row['时间']
            duration += 1
        else:
            # 结束当前事件并开始一个新的事件
            events.append((day, start_time, duration))
            start_time = last_time = row['时间']
            duration = 1
    
    # 不要忘记添加最后一个事件
    if start_time is not None:
        events.append((day, start_time, duration))
    
    # 将事件转换为DataFrame并追加到处理后的数据中
    day_events_df = pd.DataFrame(events, columns=['日期', '开始时间', '持续时长/秒'])
    day_events_df['序号'] = range(1, len(day_events_df) + 1)
    processed_data = pd.concat([processed_data, day_events_df], ignore_index=True)
    del processed_data['序号']

processed_data

日期   开始时间 持续时长/秒
0     4  12520     89
1     8   2877     80
2    11   2420     77
3    15   3165     84
4    26   1465     73
..  ...    ...    ...
59  345  11039     97
60  345  18998      5
61  345  19014     73
62  352   9662     85
63  353  26458     90

[64 rows x 3 columns]

In [12]:

# 统计每个月的故障总次数、最长和最短的持续时长
monthly_stats = processed_data.groupby('日期').agg(
    故障总次数=('开始时间', 'count'),
    最长持续时长=('持续时长/秒', 'max'),
    最短持续时长=('持续时长/秒', 'min')
)

monthly_stats


故障总次数 最长持续时长 最短持续时长
日期                      
4        1     89     89
8        1     80     80
11       1     77     77
15       1     84     84
26       2     73      3
29       1     92     92
40       1     66     66
43       1     70     70
53       1     69     69
60       3     75     10
75       1     69     69
78       1      5      5
88       1     98     98
103      1      9      9
109      1     67     67
115      1     99     99
116      1     90     90
127      2     75      4
135      1     82     82
137      1     69     69
148      1     94     94
151      1     14     14
152      1     92     92
157      2     95      9
158      1     99     99
163      1     90     90
169      1     94     94
172      2     52      9
176      1     88     88
187      1     96     96
190      1     75     75
191      1     92     92
201      1     72     72
206      1     87     87
219      1     68     68
220      1     67     67
229      2     90     88
232      1     10     10
233      1     74     74
248      1      2      2
249      1     71     71
250      1     96     96
257      1     70     70
260      1     13     13
270      1     86     86
277      1     95     95
297      1     65     65
303      1     72     72
305      1      8      8
312      1     71     71
327      1     91     91
341      1     78     78
345      3     97      5
352      1     85     85
353      1     90     90

In [13]:
shapemax=0
temp=pd.DataFrame()
temp1 = pd.DataFrame()
for d in  ['1001','2001','4001','4002','4003','5001','5002','6001','6002']:
    a=data[data[d]==1]
    # 根据用户的明确说明，连续的时间值表示持续的故障事件，
    # 我们应该相应地将这些事件分组来计算持续时间。

    # 提取唯一的日期，以便分别处理每天的数据
    days = a['日期'].unique()

    # 初始化一个空的DataFrame，用于存储处理后的结果
    processed_data = pd.DataFrame(columns=['序号', '日期', '开始时间', '持续时长/秒'])

    # 对每一天的数据进行处理，计算连续事件的持续时间
    for day in days:
        # 筛选出当前天的数据，并按时间排序
        day_data = a[a['日期'] == day].sort_values(by='时间')

        # 初始化变量，用于跟踪连续事件
        start_time = None  # 开始时间
        last_time = None   # 上一个时间点
        duration = 0       # 持续时间

        # 用于存储当天事件的列表
        events = []

        for index, row in day_data.iterrows():
            if start_time is None:
                # 开始一个新的事件
                start_time = last_time = row['时间']
                duration = 1
            elif row['时间'] == last_time + 1:
                # 继续当前的事件
                last_time = row['时间']
                duration += 1
            else:
                # 结束当前事件并开始一个新的事件
                events.append((day, start_time, duration))
                start_time = last_time = row['时间']
                duration = 1

        # 不要忘记添加最后一个事件
        if start_time is not None:
            events.append((day, start_time, duration))

        # 将事件转换为DataFrame并追加到处理后的数据中
        day_events_df = pd.DataFrame(events, columns=['日期', '开始时间', '持续时长/秒'])
        day_events_df['序号'] = range(1, len(day_events_df) + 1)
        processed_data = pd.concat([processed_data, day_events_df], ignore_index=True)
    if shapemax<processed_data.shape[0]:
        shapemax=processed_data.shape[0]
    del processed_data['序号']
    print(temp.shape)
    temp=pd.concat([temp, processed_data],axis=1)
            # 统计每个月的故障总次数、最长和最短的持续时长
    monthly_stats = processed_data.groupby('日期').agg(
        故障总次数=('开始时间', 'count'),
        最长持续时长=('持续时长/秒', 'max'),
        最短持续时长=('持续时长/秒', 'min')
    )
    monthly_stats['month'] = monthly_stats.index // 31 + 1
    monthly_stats1 = monthly_stats.groupby('month').agg(
        故障总次数=('故障总次数', 'count'),
        最长持续时长=('最长持续时长', 'max'),
        最短持续时长=('最短持续时长', 'min')
    )
    temp1=pd.concat([temp1, monthly_stats1],axis=1)
    print(d)
    display(monthly_stats1)
temp1

(0, 0)
1001


故障总次数 最长持续时长 最短持续时长
month                     
1          6     92      3
2          4     75     10
3          3     98      5
4          4     99      9
5          6     94      4
6          6     99      9
7          5     96     72
8          5     90     10
9          7     96      2
10         3     72      8
11         2     91     71
12         4     97      5

(64, 3)
2001


故障总次数 最长持续时长 最短持续时长
month                     
1         22      7      1
2         23     10      1
3         19      7      1
4         23     16      1
5         21     10      1
6         23     10      1
7         22     61      1
8         22      7      1
9         23      7      1
10        20      7      1
11        23      8      1
12        16      9      1

(3268, 6)
4001


...

(8987, 21)
6001


故障总次数 最长持续时长 最短持续时长
month                     
1         22     11      1
2         23     11      1
3         21      8      1
4         23     11      1
5         21     11      1
6         23     11      1
7         22     11      1
8         22      4      1
9         23      5      1
10        21     11      1
11        23      5      1
12        16      8      1

(8987, 24)
6002


故障总次数 最长持续时长 最短持续时长
month                     
1         11     77      1
2         18     10      1
3         12     17      1
4         17     11      1
5         16     11      1
6         13     11      1
7         18     93      1
8         15     11      1
9         17     66      1
10        17      8      1
11        10     11      1
12        12     10      1

故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  \
month                                                                        
1         6      92      3    22       7      1   1.0      69      1    12   
2         4      75     10    23      10      1   3.0      98      1    17   
3         3      98      5    19       7      1   2.0      90      1    11   
4         4      99      9    23      16      1   1.0      93      1    18   
5         6      94      4    21      10      1   NaN     NaN    NaN    14   
6         6      99      9    23      10      1   2.0      69      1    13   
7         5      96     72    22      61      1   2.0      76      1    18   
8         5      90     10    22       7      1   2.0      88      1    16   
9         7      96      2    23       7      1   2.0      74      1    18   
10        3      72      8    20       7      1   3.0      76      1    16   
11        2      91     71    23       8      1   2.0     100      1    11   
12        4      97      5    16       9      1   3.0      84      1    12   

       ... 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长 最短持续时长 故障总次数  最长持续时长  \
month  ...                                                                  
1      ...      1    22      89      1    22      11      1    11      77   
2      ...      1    23      11      1    23      11      1    18      10   
3      ...      1    21      71      1    21       8      1    12      17   
4      ...      1    23      11      1    23      11      1    17      11   
5      ...      1    21      11      1    21      11      1    16      11   
6      ...      1    23      11      1    23      11      1    13      11   
7      ...      1    22      16      1    22      11      1    18      93   
8      ...      1    22      14      1    22       4      1    15      11   
9      ...      1    23      11      1    23       5      1    17      66   
10     ...      1    21      28      1    21      11      1    17       8   
11     ...      1    23      11      1    23       5      1    10      11   
12     ...      1    16      11      1    16       8      1    12      10   

      最短持续时长  
month         
1          1  
2          1  
3          1  
4          1  
5          1  
6          1  
7          1  
8          1  
9          1  
10         1  
11         1  
12         1  

[12 rows x 27 columns]

In [14]:
# 新建一列名为“示例列”，并随机赋予一些值
temp.insert(0, '序号', range(1, shapemax+ 1))
temp

序号   日期   开始时间 持续时长/秒   日期   开始时间 持续时长/秒   日期   开始时间 持续时长/秒  ...  \
0        1    4  12520     89    1  20155      1   23  19179     69  ...   
1        2    8   2877     80    1  23502      1   23  19258      1  ...   
2        3   11   2420     77    1  25118      1   31  13465     98  ...   
3        4   15   3165     84    2  20155      1   31  13573      1  ...   
4        5   26   1465     73    3  10166      1   39  19102     71  ...   
...    ...  ...    ...    ...  ...    ...    ...  ...    ...    ...  ...   
8982  8983  NaN    NaN    NaN  NaN    NaN    NaN  NaN    NaN    NaN  ...   
8983  8984  NaN    NaN    NaN  NaN    NaN    NaN  NaN    NaN    NaN  ...   
8984  8985  NaN    NaN    NaN  NaN    NaN    NaN  NaN    NaN    NaN  ...   
8985  8986  NaN    NaN    NaN  NaN    NaN    NaN  NaN    NaN    NaN  ...   
8986  8987  NaN    NaN    NaN  NaN    NaN    NaN  NaN    NaN    NaN  ...   

     持续时长/秒   日期   开始时间 持续时长/秒   日期   开始时间 持续时长/秒   日期   开始时间 持续时长/秒  
0         1    1   4484      1    1  21945      2    1  22725      2  
1         1    1  10244      1    1  22467      1    1  22865      2  
2         1    1  10563      1    1  25147      1    1  22965      2  
3         1    1  12603      1    1  25583      1    1  23145      2  
4         1    1  13444      1    1  27227      1    1  23523      4  
...     ...  ...    ...    ...  ...    ...    ...  ...    ...    ...  
8982    NaN  362  21933      8  NaN    NaN    NaN  NaN    NaN    NaN  
8983    NaN  362  21951      1  NaN    NaN    NaN  NaN    NaN    NaN  
8984    NaN  362  25791      1  NaN    NaN    NaN  NaN    NaN    NaN  
8985    NaN  362  28239      2  NaN    NaN    NaN  NaN    NaN    NaN  
8986    NaN  362  28251      1  NaN    NaN    NaN  NaN    NaN    NaN  

[8987 rows x 28 columns]

In [18]:
temp.to_excel('result2.xlsx',index=None,writer,sheet_name='M202')

In [28]:
temp1 = pd.read_excel('result2_M201.xlsx')

In [29]:
with pd.ExcelWriter('result2.xlsx', engine='openpyxl') as writer:
    temp1.to_excel(writer,sheet_name='M201')
    temp.to_excel(writer,sheet_name='M202')

In [15]:
temp1.to_excel('result2_M202_month.xlsx',sheet_name='M202')